In [1]:
import os
import pycocotools.coco as coco

from pycocotools.cocoeval import COCOeval

In [2]:
split = 'val'
annot_path = os.path.join('F:\TUM Learning Material\Forschung\CenterNet\CenterNet-master\data\coco', 'annotations','instances_{}2017.json').format(split)
#initialize coco object given file_path
coco = coco.COCO(annot_path)

centernet_dets = coco.loadRes('{}/results.json'.format('F:\TUM Learning Material\Forschung\CenterNet\CenterNet-master\exp\ctdet\hourglass'))

loading annotations into memory...
Done (t=1.05s)
creating index...
index created!
Loading and preparing results...
DONE (t=4.02s)
creating index...
index created!


In [3]:
coco_eval = COCOeval(coco, centernet_dets, "bbox")

In [4]:
coco_eval.evaluate()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=57.78s).


In [5]:
result = coco_eval.accumulate()

Accumulating evaluation results...
DONE (t=14.58s).


In [6]:
result['counts']

[10, 101, 80, 4, 3]

In [7]:
result['precision'].shape

(10, 101, 80, 4, 3)

In [8]:
coco_eval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.591
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.440
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.532
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.531
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.557
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.602
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.704


In [9]:
T, R, K, A, M = result['counts']

In [10]:
precision_matrix = result['precision']

In [11]:
import numpy as np

In [12]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_precision(precision_matrix, area_idx, maxDet_idx, T=10, R=101, K=80):
    class_precision = precision_matrix[:,:,:,area_idx,maxDet_idx]
    class_precision = class_precision[class_precision>-1].reshape(T,R,K)
    class_precision = np.mean(class_precision, axis=0)
    class_precision = np.mean(class_precision, axis=0)
    return np.round(class_precision, decimals=3) 

In [19]:
smallarea_precision = get_class_precision(precision_matrix, area_idx=1, maxDet_idx=1)
middlearea_precision = get_class_precision(precision_matrix, area_idx=2, maxDet_idx=1)
largearea_precision = get_class_precision(precision_matrix, area_idx=3, maxDet_idx=1)
total_precision = get_class_precision(precision_matrix, area_idx=0, maxDet_idx=1)

In [20]:
class_name = ['person', 'bicycle', 'car', 'motorcycle', 'airplane','bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
      'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
      'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
      'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
      'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
      'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
      'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
      'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
      'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
      'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
      'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
      'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [21]:
import sys
from prettytable import PrettyTable

In [22]:
class_precision_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_precision_table.add_row([class_name[cat],smallarea_precision[cat],middlearea_precision[cat],largearea_precision[cat],total_precision[cat]])

In [23]:
print(class_precision_table)

+----------------+-------+--------+-------+-------+
|     class      | small | middle | large | total |
+----------------+-------+--------+-------+-------+
|     person     | 0.286 |  0.58  | 0.725 | 0.502 |
|    bicycle     | 0.134 | 0.353  | 0.567 | 0.289 |
|      car       | 0.332 | 0.596  | 0.672 | 0.436 |
|   motorcycle   | 0.207 | 0.379  | 0.571 | 0.407 |
|    airplane    | 0.635 | 0.596  | 0.575 |  0.59 |
|      bus       | 0.293 | 0.479  | 0.765 | 0.621 |
|     train      | 0.291 | 0.277  |  0.62 | 0.573 |
|     truck      | 0.201 | 0.366  | 0.525 | 0.369 |
|      boat      | 0.171 | 0.317  | 0.442 |  0.26 |
| traffic light  | 0.184 | 0.449  | 0.469 | 0.234 |
|  fire hydrant  | 0.355 | 0.748  | 0.811 | 0.677 |
|   stop sign    | 0.261 | 0.733  | 0.942 | 0.669 |
| parking meter  | 0.079 | 0.451  | 0.759 | 0.462 |
|     bench      | 0.142 | 0.263  | 0.371 | 0.241 |
|      bird      | 0.185 | 0.572  | 0.749 | 0.319 |
|      cat       | 0.259 | 0.552  | 0.647 | 0.622 |
|      dog  

In [24]:
precision_sort_idx = np.argsort(-total_precision, kind='mergesort')
i = 0
for i in range(len(precision_sort_idx)):
    print(class_name[precision_sort_idx[i]])

bear
fire hydrant
stop sign
giraffe
zebra
frisbee
cat
bus
dog
mouse
elephant
toilet
laptop
airplane
horse
train
microwave
tv
refrigerator
skateboard
tennis racket
person
keyboard
cow
sheep
clock
parking meter
pizza
couch
sports ball
car
cup
motorcycle
teddy bear
suitcase
donut
sink
umbrella
surfboard
truck
bowl
fork
kite
bottle
wine glass
baseball glove
cell phone
cake
toaster
vase
tie
bed
snowboard
oven
bird
remote
sandwich
chair
bicycle
baseball bat
hot dog
boat
skis
toothbrush
scissors
orange
potted plant
bench
traffic light
carrot
knife
banana
spoon
broccoli
handbag
backpack
dining table
apple
book
hair drier


In [32]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_recall(recall_matrix, area_idx, maxDet_idx, T=10, K=80):
    recall_matrix = recall_matrix[:,:,area_idx,maxDet_idx]
    recall_matrix = recall_matrix[recall_matrix>-1].reshape(T,K)
    recall_matrix = np.mean(recall_matrix, axis=0)
    return np.round(recall_matrix, decimals=3) 

In [29]:
recall_matrix = result['recall']

In [30]:
recall_matrix.shape

(10, 80, 4, 3)

In [33]:
smallarea_recall = get_class_recall(recall_matrix, area_idx=1, maxDet_idx=1)
middlearea_recall = get_class_recall(recall_matrix, area_idx=2, maxDet_idx=1)
largearea_recall = get_class_recall(recall_matrix, area_idx=3, maxDet_idx=1)
total_recall = get_class_recall(recall_matrix, area_idx=0, maxDet_idx=1)

In [35]:
class_recall_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_recall_table.add_row([class_name[cat],smallarea_recall[cat],middlearea_recall[cat],largearea_recall[cat],total_recall[cat]])

In [37]:
print(class_recall_table)

+----------------+-------+--------+-------+-------+
|     class      | small | middle | large | total |
+----------------+-------+--------+-------+-------+
|     person     | 0.345 | 0.632  | 0.795 | 0.559 |
|    bicycle     | 0.225 |  0.48  | 0.644 | 0.391 |
|      car       | 0.415 | 0.684  | 0.784 | 0.526 |
|   motorcycle   | 0.323 | 0.493  | 0.662 | 0.513 |
|    airplane    | 0.727 |  0.67  | 0.663 | 0.678 |
|      bus       | 0.364 |  0.59  | 0.824 | 0.698 |
|     train      | 0.375 | 0.456  | 0.714 | 0.682 |
|     truck      | 0.475 | 0.641  | 0.727 | 0.622 |
|      boat      | 0.271 | 0.423  | 0.598 |  0.37 |
| traffic light  | 0.325 | 0.575  | 0.616 | 0.373 |
|  fire hydrant  | 0.544 | 0.787  | 0.855 | 0.751 |
|   stop sign    | 0.567 | 0.871  | 0.956 | 0.792 |
| parking meter  | 0.312 | 0.624  | 0.813 | 0.613 |
|     bench      | 0.252 | 0.433  | 0.572 | 0.401 |
|      bird      | 0.247 | 0.654  | 0.842 | 0.395 |
|      cat       |  0.55 | 0.708  | 0.773 | 0.756 |
|      dog  

In [38]:
recall_sort_idx = np.argsort(-total_recall, kind='mergesort')
i = 0
for i in range(len(recall_sort_idx)):
    print(class_name[recall_sort_idx[i]])

bear
stop sign
cat
fire hydrant
toilet
dog
giraffe
frisbee
zebra
microwave
bus
elephant
laptop
train
mouse
airplane
refrigerator
horse
tv
keyboard
couch
truck
pizza
parking meter
skateboard
clock
tennis racket
toaster
cow
bed
bowl
cup
sheep
person
teddy bear
sandwich
cell phone
sink
oven
car
sports ball
vase
motorcycle
fork
umbrella
suitcase
bottle
donut
cake
remote
surfboard
snowboard
kite
wine glass
baseball glove
potted plant
chair
tie
orange
hot dog
scissors
toothbrush
spoon
bench
bird
bicycle
dining table
baseball bat
skis
knife
traffic light
boat
carrot
handbag
backpack
apple
broccoli
banana
hair drier
book
